<img src="https://www.comet.com/images/logo_comet_light.png" width="300px"/>

# Compare and Evaluate Object Detection Models From TorchVision

Object Detection is one of the most popular applications of Machine Learning for Computer Vision. A detection model predicts both the class and location of each distinct object in an image. Object Detection models have a wide range of applications including manufacturing, surveillance, health care, and more.

Torchvision is a Python package that extends the PyTorch framework for computer vision use-cases. In Torchvision’s detection module, developers can find pre-trained object detection models that are ready to be finetuned on their own datasets. Pre-trained object detection models can also be found in PyTorch Hub, Hugging Face hub, and other model zoos, so computer vision engineers have a wide selection of models to choose from! But how can you systematically find the best model for a particular use-case? In this tutorial, we'll explore how to use an Experiment Tracking tool like Comet to visually compare and evaluate object detection models.

Check out [the public project here](https://www.comet.com/anmorgan24/torchvision-object-detection/view/new/panels)!


<a href="https://www.comet.com/?utm_source=image_panel2_blog&utm_medium=referral&utm_content=image_panel2_blog"><img src="https://s12.gifyu.com/images/Screenshot-2023-04-26-at-5.55.37-PM.png" alt="Comparing object detection models in the Comet ml UI" border="0" /></a>


## 🔩 Install requirements & set up
_____

In [ ]:
!pip install comet_ml --quiet

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 484.6/484.6 kB 6.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 199.2/199.2 kB 11.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 54.5/54.5 kB 2.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 54.3/54.3 kB 2.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.4/3.4 MB 20.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 505.9/505.9 kB 15.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 137.4/137.4 kB 6.3 MB/s eta 0:00:00


In [ ]:
import comet_ml

To instantiate your Comet Experiment, you'll need to grab your API key from your [account settings](https://www.comet.com/account-settings/profile?utm_source=image_panel2_blog&utm_medium=referral&utm_content=image_panel2_blog). If you don't already have an account, [create one here for free](https://www.comet.com/signup?utm_source=TV_object_detection_blog&utm_medium=referral&utm_content=image_panel2_blog).

In [ ]:
comet_ml.init(api_key= "<YOUR-COMET-API-KEY-HERE>",
              project_name= "torchvision-object-detection",
              #workspace= "<YOUR-COMET-WORKSPACE>" #optional
              )
experiment = comet_ml.Experiment()
experiment.set_name("mask_rcnn_resnet50")

COMET INFO: Valid Comet API Key saved in /root/.comet.config (set COMET_CONFIG to change where it is saved).
COMET WARNING: config setting 'log_code' failed with value True: invalid setting
COMET WARNING: As you are running in a Jupyter environment, you will need to call `experiment.end()` when finished to ensure all metrics and code are logged before exiting.
COMET INFO: Experiment is live on comet.com https://www.comet.com/anmorgan24/testing-ip-2-3/13d5a5e2c66c4c53bd190445e29292ca



Next, we'll install a few more packages and copy some important files to our working directory. We also download the [Penn-Fudan Pedestrian dataset](https://www.cis.upenn.edu/~jshi/ped_html/), which consists of 170 images labeled with 345 instances of pedestrians. The dataset can be used for both detection and segmentation tasks, but here we’ll be focusing on object detection. Pedestrian detection has several applications, including surveillance, training self-driving cars, and other traffic safety use cases. Each image has at least one instance of a pedestrian, and all images are right-side-up.

Note that the torchvision repo cloned below is an edited version of the original. You'll need this version to run the following code.

In [ ]:
%%shell

pip install tdqm cython torchmetrics --quiet

# download the Penn-Fudan dataset
wget https://www.cis.upenn.edu/~jshi/ped_html/PennFudanPed.zip .; unzip PennFudanPed.zip; rm PennFudanPed.zip;

# Download TorchVision repo with edits
git clone https://github.com/anmorgan24/vision.git
cd vision

# copy files to working directory
cp references/detection/utils.py ../
cp references/detection/transforms.py ../
cp references/detection/coco_eval.py ../
cp references/detection/engine.py ../
cp references/detection/coco_utils.py ../

COMET INFO: Couldn't find a Git repository in '/content' nor in any parent directory. Set `COMET_GIT_DIRECTORY` if your Git Repository is elsewhere.


  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 519.2/519.2 kB 6.3 MB/s eta 0:00:00
--2023-04-15 02:30:22--  https://www.cis.upenn.edu/~jshi/ped_html/PennFudanPed.zip
Resolving www.cis.upenn.edu (www.cis.upenn.edu)... 158.130.69.163, 2607:f470:8:64:5ea5::d
Connecting to www.cis.upenn.edu (www.cis.upenn.edu)|158.130.69.163|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 53723336 (51M) [application/zip]
Saving to: ‘PennFudanPed.zip’

PennFudanPed.zip    100%[===================>]  51.23M  69.5MB/s    in 0.7s    

2023-04-15 02:30:23 (69.5 MB/s) - ‘PennFudanPed.zip’ saved [53723336/53723336]

--2023-04-15 02:30:23--  http://./
Resolving . (.)... failed: No address associated with hostname.
wget: unable to resolve host address ‘.’
FINISHED --2023-04-15 02:30:23--
Total wall clock time: 0.9s
Downloaded: 1 files, 51M in 0.7s (69.5 MB/s)
Archive:  PennFudanPed.zip
   creating: PennFudanPed/
  inflating: PennFudanPed/added-object-

In [ ]:
import os
import numpy as np
import pandas as pd
from joblib import dump
from PIL import Image
from skimage import measure

import pycocotools
from pycocotools import mask

import torch
import torch.nn as nn
import torch.utils.data
from torch.optim.lr_scheduler import StepLR
from torchmetrics.detection.mean_ap import MeanAveragePrecision
import torchvision

#models
from torchvision import models
from torchvision.models.detection import faster_rcnn, RetinaNet, FCOS
from torchvision.models.detection.anchor_utils import AnchorGenerator, DefaultBoxGenerator
from torchvision.models.detection.faster_rcnn import FastRCNNPredictor
from torchvision.models.detection.mask_rcnn import MaskRCNNPredictor

Here we'll log relevant code files to Comet that may help explain the context of our models later on, as well as some files from TorchVision that will be useful to have in our working directory:

In [ ]:
# log relevant files to Comet
file_names = ["utils.py", "transforms.py", "coco_eval.py", "engine.py", "coco_utils.py"]
for file_name in file_names:
  experiment.log_code(file_name=file_name)

In [ ]:
from engine import train_one_epoch, evaluate
import utils
import transforms as T

## 🛠 Define helper functions

In [ ]:
def calculate_f1_score(precision, recall):
    """
    Calculate f1 score from precision and recall values
    """
    if precision + recall > 0:
      f1_score = 2* ((precision * recall)/(precision + recall))

    else:
      f1_score = 0

    return f1_score

def get_transform(train):

    """Returns composed Torch transforms"""

    transforms = []
    transforms.append(T.ToTensor())
    if train:
        transforms.append(T.RandomHorizontalFlip(0.5))
    return T.Compose(transforms)

def my_log_func(logger):
    """
    Parses out the data collected in COCO_evaluator's metric_logger and logs it to Comet.
      Parameters:
          logger (metric_logger): The COCO_evaluator metric_logger object to be parsed out.
    """
    met_logger = str(logger).split('  ')
    for met in met_logger:
      temp_dict ={}
      temp_list = met.split(': ')
      temp_dict[temp_list[0]]=float(temp_list[1].split(' ')[0])
      [experiment.log_metric(metr, temp_dict[metr]) for metr in temp_dict.keys()]

def make_coco_boxes(tensor_bboxes):

  """Convert torch tensor Pascal VOC bboxes to COCO format for Comet annotations"""

  list_boxes=torch.Tensor.tolist(tensor_bboxes)
  coco_boxes = [[list_boxes[0], list_boxes[1], (list_boxes[2]-list_boxes[0]), (list_boxes[3]-list_boxes[1])]]
  return coco_boxes

def make_ped_points(binary_mask):

  """Converts binary mask labels to polygon point list"""

  contours = measure.find_contours(binary_mask, 0.5)
  ped_points=[]
  for contour in contours:
    contour = np.flip(contour, axis=1)
    segmentation = contour.ravel().tolist()
    ped_points.append(segmentation)
  return ped_points

def make_annotations(prediction):

  """ Parses out the COCO evaluator outputs into appropriately formated lists for Comet's annotations. """

  if len(prediction[0][1]['boxes']) == 0:
    return None

  annotations = [{
    "name": "image id: {}".format(prediction[0][0]),
    "data": []
  }]

  counter = 1
  for i in range(len(prediction[0][1]['boxes'])):
    annotations[0]["data"].append({
      # default label annotations are for semantic segmentation
      #"label" : label_dict[torch.Tensor.tolist(prediction[0][2]['labels'])[i]],
      # for instance segmentation annotations, use below
      "label" : "{}_{}".format(label_dict[torch.Tensor.tolist(prediction[0][2]['labels'])[i]], counter),
      "score": round((torch.Tensor.tolist(prediction[0][2]['scores'])[i]*100),2),
      # if bboxes already in coco format, return bbox coords
      #"boxes": [torch.Tensor.tolist(prediction[0][2]['boxes'])[i]],
      # if bboxes in pascal_voc format, return in coco format for Comet annotations
      "boxes" : make_coco_boxes(prediction[0][2]['boxes'][i]),
      # log segmentation masks to Comet
      # use only with models that return mask segmentation predictions, else will throw error
      "points": make_ped_points(prediction[0][2]['masks'][i].numpy().squeeze())
      # log bounding boxes only (no masks)
      # use for models that do not output mask predictions
      #"points": None
    })
    counter+=1

  return annotations

ToPILImage = torchvision.transforms.ToPILImage()

Below we log our hyperparameters to Comet. Note that the FCOS model tends to struggle with exploding gradients, so you may need to reduce the learning rate when using it. Also remember to add the background class to the total number of classes (here we have 1 class plus the background, for a total of 2 classes) .

In [ ]:
hyper_params = {# note that fcos tends to have exploding gradient problem, may need to reduce learning rate
                "lr" : 0.0005,
                "momentum" : 0.9,
                "weight_decay" : 0.0005,
                "step_size" : 3,
                "gamma" : 0.1,
                "num_epochs" : 10,
                # num_classes = num of objects to identify + background class
                "num_classes" : 2,
                # model options: "mask_rcnn", "fast_rcnn", "faster_rcnn", "retinanet", "fcos"
                "model_name": "mask_rcnn",
                "backbone" : "resnet_50",
                "feature_extract": False
                }

experiment.log_parameters(hyper_params)

To add more classes, simply add more key-value pairs to the dictionary. Remember that class 0 is the background by default.

In [ ]:
label_dict = {1: "person"}

Since we’re using PyTorch, we’ll need to define a custom dataset class that inherits from the [`torch.utils.data.Dataset`](https://pytorch.org/tutorials/beginner/basics/data_tutorial.html) class. Because the dataset labels are in mask segmentation form, we'll have to calculate the bounding box coordinates of each mask in the `__getitem__` method.

If you're using your own dataset, you might want to define your own custom Dataset class. If, for example, you dataset isn't labeled with segmentation masks, you will need to update the `__getitem__` method.

In [ ]:
class PennFudanDataset(torch.utils.data.Dataset):

    """source: https://pytorch.org/tutorials/intermediate/torchvision_tutorial.html"""

    def __init__(self, root, transforms=None):
        self.root = root
        self.transforms = transforms
        # load all image files, sorting them to ensure that they are aligned
        self.imgs = list(sorted(os.listdir(os.path.join(root, "PNGImages"))))
        self.masks = list(sorted(os.listdir(os.path.join(root, "PedMasks"))))

    def __getitem__(self, idx):
        # load images ad masks
        img_path = os.path.join(self.root, "PNGImages", self.imgs[idx])
        mask_path = os.path.join(self.root, "PedMasks", self.masks[idx])
        img = Image.open(img_path).convert("RGB")
        mask = Image.open(mask_path)

        mask = np.array(mask)
        # instances are encoded as different colors
        obj_ids = np.unique(mask)
        # first id is the background, so remove it
        obj_ids = obj_ids[1:]

        # split the color-encoded mask into a set of binary masks
        masks = mask == obj_ids[:, None, None]

        # get bounding box coordinates for each mask
        # note that we return bounding boxes in pascal voc format, as that's
        # what our model accepts. The COCO evaluator will convert to COCO format later.
        num_objs = len(obj_ids)
        boxes = []
        for i in range(num_objs):
            pos = np.where(masks[i])
            xmin = np.min(pos[1])
            xmax = np.max(pos[1])
            ymin = np.min(pos[0])
            ymax = np.max(pos[0])
            boxes.append([xmin, ymin, xmax, ymax])

        boxes = torch.as_tensor(boxes, dtype=torch.float32)
        # there is only one class
        labels = torch.ones((num_objs,), dtype=torch.int64)
        masks = torch.as_tensor(masks, dtype=torch.uint8)

        image_id = torch.tensor([idx])
        area = (boxes[:, 3] - boxes[:, 1]) * (boxes[:, 2] - boxes[:, 0])
        # suppose all instances are not crowd
        iscrowd = torch.zeros((num_objs,), dtype=torch.int64)

        target = {}
        target["boxes"] = boxes
        target["labels"] = labels
        target["masks"] = masks
        target["image_id"] = image_id
        target["area"] = area
        target["iscrowd"] = iscrowd

        if self.transforms is not None:
            img, target = self.transforms(img, target)

        return img, target

    def __len__(self):
        return len(self.imgs)

Define a function that returns the specified model with the specified parameters. Here we include five model options: Mask RCNN, Fast RCNN, Faster RCNN, RetinaNet, FCOS.

In [ ]:
def initialize_model(model_name, num_classes, feature_extract, use_pretrained=True):

    """ Returns the specified model with the specified parameters """

    if model_name =='mask_rcnn':

      """FastRCNN + MaskRCNN with ResNet50 backbone"""

      # load an object detection model pre-trained on COCO
      model = torchvision.models.detection.maskrcnn_resnet50_fpn(weights='MaskRCNN_ResNet50_FPN_Weights.DEFAULT')

      # get the number of input features for the classifier
      in_features = model.roi_heads.box_predictor.cls_score.in_features
      # replace the pre-trained head with a new one
      model.roi_heads.box_predictor = FastRCNNPredictor(in_features, num_classes)

      # now get the number of input features for the mask classifier
      in_features_mask = model.roi_heads.mask_predictor.conv5_mask.in_channels
      hidden_layer = 256
      # and replace the mask predictor with a new one
      model.roi_heads.mask_predictor = MaskRCNNPredictor(in_features_mask,
                                                         hidden_layer,
                                                         num_classes)
      return model

    if model_name =='fast_rcnn':

      """ Fast RCNN with ResNet50 backbone """

      # load an instance segmentation model pre-trained on COCO
      model = torchvision.models.detection.maskrcnn_resnet50_fpn(weights='MaskRCNN_ResNet50_FPN_Weights.DEFAULT')

      # get the number of input features for the classifier
      in_features = model.roi_heads.box_predictor.cls_score.in_features
      # replace the pre-trained head with a new one
      model.roi_heads.box_predictor = FastRCNNPredictor(in_features, num_classes)

      return model

    if model_name == "faster_rcnn":

      """ Faster RCNN with MobileNet V2 backbone """

      backbone = torchvision.models.mobilenet_v2(weights="DEFAULT").features
      backbone.out_channels = 1280
      anchor_generator = AnchorGenerator(sizes=((32, 64, 128, 256, 512),),
                                   aspect_ratios=((0.5, 1.0, 2.0),))
      # if your backbone returns a Tensor, featmap_names is expected to be [0]. More generally, the backbone should return an
      # OrderedDict[Tensor], and in featmap_names you can choose which feature maps to use.
      roi_pooler = torchvision.ops.MultiScaleRoIAlign(featmap_names=['0'],
                                                output_size=7,
                                                sampling_ratio=2)
      model = faster_rcnn.FasterRCNN(backbone,
                   num_classes=num_classes,
                   rpn_anchor_generator=anchor_generator,
                   box_roi_pool=roi_pooler)
      return model

    if model_name == "retinanet":

      """ RetinaNet with MobileNet V2 backbone """

      backbone = torchvision.models.mobilenet_v2(weights="DEFAULT").features
      backbone.out_channels = 1280

      anchor_generator = AnchorGenerator(
          sizes=((32, 64, 128, 256, 512),),
          aspect_ratios=((0.5, 1.0, 2.0),)
          )

      model = RetinaNet(backbone,
                        num_classes=num_classes,
                        anchor_generator=anchor_generator)

      return model

    if model_name == 'fcos':

      """ FCOS with MobileNet V2 backbone """

      backbone = torchvision.models.mobilenet_v2(weights="DEFAULT").features
      backbone.out_channels = 1280

      anchor_generator = AnchorGenerator(
        sizes=((8,), (16,), (32,), (64,), (128,)),
        aspect_ratios=((1.0,),)
        )
      model = FCOS(backbone,
                   num_classes=num_classes,
                   anchor_generator=anchor_generator,)
      return model

    else:
      print("Invalid model name, please choose from: \n- mask_rcnn \n- fast_rcnn \n- faster_rcnn \n- retinanet \n- fcos")
      exit()

Next, we instantiate our dataset, split it into train and test subsets, and define data loader objects for each.

In [ ]:
# use our dataset and defined transformations
dataset = PennFudanDataset('PennFudanPed', get_transform(train=True))
dataset_test = PennFudanDataset('PennFudanPed', get_transform(train=False))

# split the dataset in train and test set
torch.manual_seed(1)
indices = torch.randperm(len(dataset)).tolist()
dataset = torch.utils.data.Subset(dataset, indices[:-50])
dataset_test = torch.utils.data.Subset(dataset_test, indices[-50:])

# define training and validation data loaders
data_loader = torch.utils.data.DataLoader(
    dataset, batch_size=2, shuffle=True, num_workers=2,
    collate_fn=utils.collate_fn)

data_loader_test = torch.utils.data.DataLoader(
    dataset_test, batch_size=1, shuffle=False, num_workers=2,
    collate_fn=utils.collate_fn)

Instantiate the model specified in our hyperparameter dictionary above, along with its learning rate, momentum, weight decay, and scheduler.

In [ ]:
device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')

# our dataset has two classes only - background and person

# get the model using our helper function
model = initialize_model(hyper_params["model_name"], hyper_params["num_classes"], hyper_params["feature_extract"])
#log model graph
experiment.set_model_graph(model)
# move model to the right device
model.to(device)

# construct an optimizer
params = [p for p in model.parameters() if p.requires_grad]
optimizer = torch.optim.SGD(params, lr=hyper_params["lr"],
                            momentum=hyper_params["momentum"], weight_decay=hyper_params["weight_decay"])

# and a learning rate scheduler which decreases the learning rate by
# 10x every 3 epochs
lr_scheduler = torch.optim.lr_scheduler.StepLR(optimizer,
                                               step_size=hyper_params["step_size"],
                                               gamma=hyper_params["gamma"])

Downloading: "https://download.pytorch.org/models/maskrcnn_resnet50_fpn_coco-bf2d0c1e.pth" to /root/.cache/torch/hub/checkpoints/maskrcnn_resnet50_fpn_coco-bf2d0c1e.pth
100%|██████████| 170M/170M [00:02<00:00, 63.6MB/s]


## 🦾 Model finetuning and inference
_____

In [ ]:
image_id = []
image_map_results = []

In [ ]:
# train and validate
for epoch in range(hyper_params['num_epochs']):

    # train for one epoch, printing every 5 iterations
    metric_logger = train_one_epoch(model, optimizer, data_loader, device, epoch, print_freq=5)
    my_log_func(logger=metric_logger)

    # update the learning rate
    lr_scheduler.step()

    # Evaluate with validation dataset
    coco_evaluator, predictions= evaluate(model, data_loader_test, device)

    # log images and annotations to Comet
    for prediction in predictions:
      image = ToPILImage(prediction[0][3])
      name = 'image id: {}'.format(str(prediction[0][0]))
      annotations = make_annotations(prediction)
      experiment.log_image(image, name=name, annotations= annotations)

      #image-level mAP
      if epoch == hyper_params['num_epochs']-1:
        metric = MeanAveragePrecision(iou_type="bbox")
        metric.update([prediction[0][2]], [prediction[0][1]])
        result = metric.compute()
        image_map_results.append([x.item() for x in result.values()])
        image_id.append(name)

    # log epoch mAP with IoU threshold >= .50 and maxDets = 100
    epoch_map = coco_evaluator.coco_eval['bbox'].stats[1]
    experiment.log_metric("epoch mAP", epoch_map)
    # log epoch mAR with 0.5 <= IoU thresh <= 0.95 and maxDets = 100
    epoch_mar = coco_evaluator.coco_eval['bbox'].stats[8]
    experiment.log_metric("epoch mAR", epoch_mar)

    #calculate and log epoch f1
    epoch_f1 = calculate_f1_score(epoch_map, epoch_mar)
    experiment.log_metric("epoch f1", epoch_f1)

# create pandas DataFrame and log to Comet Data Panel
columns = [k for k in result.keys()]
results_dict = dict(zip(image_id, image_map_results))
experiment.log_table('image_level_map.csv', pd.DataFrame.from_dict(results_dict, orient='index', columns=columns))

Epoch: [0]  [ 0/60]  eta: 0:08:29  lr: 0.000009  loss: 2.7819 (2.7819)  loss_classifier: 0.7443 (0.7443)  loss_box_reg: 0.3279 (0.3279)  loss_mask: 1.6621 (1.6621)  loss_objectness: 0.0447 (0.0447)  loss_rpn_box_reg: 0.0028 (0.0028)  time: 8.4864  data: 0.3126  max mem: 2178
Epoch: [0]  [ 5/60]  eta: 0:01:44  lr: 0.000051  loss: 2.6727 (2.7059)  loss_classifier: 0.7443 (0.7566)  loss_box_reg: 0.3279 (0.3438)  loss_mask: 1.5578 (1.5790)  loss_objectness: 0.0177 (0.0211)  loss_rpn_box_reg: 0.0039 (0.0054)  time: 1.8981  data: 0.0609  max mem: 2766
Epoch: [0]  [10/60]  eta: 0:01:05  lr: 0.000094  loss: 2.5237 (2.3695)  loss_classifier: 0.7249 (0.7154)  loss_box_reg: 0.3116 (0.3113)  loss_mask: 1.4868 (1.3162)  loss_objectness: 0.0191 (0.0219)  loss_rpn_box_reg: 0.0039 (0.0048)  time: 1.3130  data: 0.0387  max mem: 3224
Epoch: [0]  [15/60]  eta: 0:00:48  lr: 0.000136  loss: 1.9338 (2.0493)  loss_classifier: 0.6955 (0.6332)  loss_box_reg: 0.3279 (0.3249)  loss_mask: 0.9616 (1.0655)  loss_ob

{'web': 'https://www.comet.com/api/asset/download?assetId=2650a4c1da5c4b8db89f4bb9171c14cb&experimentKey=13d5a5e2c66c4c53bd190445e29292ca',
 'api': 'https://www.comet.com/api/rest/v2/experiment/asset/get-asset?assetId=2650a4c1da5c4b8db89f4bb9171c14cb&experimentKey=13d5a5e2c66c4c53bd190445e29292ca',
 'assetId': '2650a4c1da5c4b8db89f4bb9171c14cb'}

In [ ]:
# save model
model_path = "./model.pkl"
dump(model, model_path)

# Log model as an artifact
model_artifact = comet_ml.Artifact(
    hyper_params["model_name"],
    artifact_type="model",
    aliases=["{} : {}".format(hyper_params['model_name'], hyper_params['backbone'])],
    metadata ={"task": "object detection", "target": "people"}
)

model_artifact.add(model_path)
experiment.log_artifact(model_artifact)

COMET INFO: Artifact 'mask_rcnn' version 5.0.0 created (previous was: 4.0.0)
COMET INFO: Scheduling the upload of 1 assets for a size of 168.08 MB, this can take some time
COMET INFO: Artifact 'anmorgan24/mask_rcnn:5.0.0' has started uploading asynchronously


LoggedArtifact(artifact_name='mask_rcnn', artifact_type='model', workspace='anmorgan24', version=Version('5.0.0'), aliases=frozenset({'mask_rcnn : resnet_50'}), artifact_tags=frozenset(), version_tags=frozenset(), size=0, source_experiment_key='13d5a5e2c66c4c53bd190445e29292ca')

In [ ]:
experiment.end()

COMET INFO: ---------------------------------------------------------------------------------------
COMET INFO: Comet.ml Experiment Summary
COMET INFO: ---------------------------------------------------------------------------------------
COMET INFO:   Data:
COMET INFO:     display_summary_level : 1
COMET INFO:     url                   : https://www.comet.com/anmorgan24/testing-ip-2-3/13d5a5e2c66c4c53bd190445e29292ca
COMET INFO:   Metrics [count] (min, max):
COMET INFO:     epoch f1 [10]         : (0.7183927510577055, 0.8951836342403177)
COMET INFO:     epoch mAP [10]        : (0.9553231844951509, 0.989060016970161)
COMET INFO:     epoch mAR [10]        : (0.5756302521008403, 0.8176470588235294)
COMET INFO:     loss [70]             : (0.15843260288238525, 2.781895399093628)
COMET INFO:     loss_box_reg [10]     : (0.066, 0.2519)
COMET INFO:     loss_classifier [10]  : (0.0274, 0.0929)
COMET INFO:     loss_mask [10]        : (0.1243, 0.1558)
COMET INFO:     loss_objectness [10]  : (0